## The 2017 Eagle Creek Fire near Hood River, Oregon covered more than 48,000 acres and devastated more than half of the Oregon side of the Columbia River Gorge.

![3D extent of Eagle Creek Fire](https://wildfiretoday.com/wp-content/uploads/2017/09/EagleCr_3D2_730pmPDT_9-5-2017.jpg)

Image Source: [Wildfire Today](https://wildfiretoday.com/wp-content/uploads/2017/09/EagleCr_3D2_730pmPDT_9-5-2017.jpg)

![Mountains on Fire](https://advancelocal-adapter-image-uploads.s3.amazonaws.com/image.oregonlive.com/home/olive-media/width2048/img/wildfires/photo/eagle-creek-fire-f163f5cfde990cb7.jpg)

Image Source: [Oregon Live](https://advancelocal-adapter-image-uploads.s3.amazonaws.com/image.oregonlive.com/home/olive-media/width2048/img/wildfires/photo/eagle-creek-fire-f163f5cfde990cb7.jpg)

On a hot and windy fall afternoon on September 2, 2017, two young teens wrecklessly playing with fireworks ignited one of the largest wildfires in northern Oregon's history, descimating near half of the Columbia River Gorge, a famous outdoor recreation area and critical transportation corridor (trains, trucks, and boats).

As the fire quickly grew, 152 hikers were trapped by the flames and eventually rescued while the nearby town of Cascade Locks was evacuated. 2.5 days after ignition, the fire had grown to 10,000 acres and traveled nearly 13 miles in approximately 16 hours, fueled by strong winds. By this time, Interstate 84 has been shutdown and the fire jumped an incredible ~1800 ft across the Columbia River and spread into parts of Washington. Through the blaze, nearly 400 homes were evacuated, hundreds of miles of trail and roadway infracture were destroyed, and historic landmarks, such a [Multinomah Falls](https://www.fs.usda.gov/recarea/crgnsa/recarea/?recid=30026), closely threatened.

The fire was finally contained at the end of November, nearly two months after it started, but continued to smoulder for another 9-months. A total of 48,000 acres burned in the Eagle Creek Fire. While residents were eventually able to return home, the Eagle Creek Trail system remained closed for three years and remains a visible scar on the landscape today. Massive efforts were put into clearing dangerous trees, installing protective fensing, and monitoring for landslide and debris flows. The teen responsible for the fire was sentenced to 1,920 hours of community service with the Forest Service. If this service is not compled in 10 years, the teen will have to pay $36.6 million in restitution.

### **2017 Eagle Creek Fire Boundary**

In [ ]:
# Import standard libraries
import logging              # Logging of messages
import os                   # Handles files and directories
import pathlib              # Provides a path class for cross platform use
import subprocess           # Run new code by creating new process
import warnings             # Show warning triggered by module import
from glob import glob       # Creates lists of files for batch processing

# Import third party libraries
import earthpy.appeears as etapp    # Plot and manipulate spatial data
import folium               # Generate interactive maps
import geopandas as gpd     # Work with vector files
import hvplot.pandas        # Used for plotting data
import hvplot.xarray        # Used for plotting maps
import numpy as np          # Used for math operations
import pandas as pd         # Used for working with tabular data
import rioxarray as rxr     # Storage and manipulation of rasters
import xarray as xr         # Work with multidimensional arrays

# Set up logging so AppeearsDownloader will log in notebook
logging.basicConfig(level=logging.INFO)

# Ignore FutureWarning coming from hvplot
#warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Define/create a directory to store and access data

eagle_prj_dir = os.path.join(
    pathlib.Path.home(), 'earth-analytics', 'data', 'camp-fire')
# This defines where the directory can be found

os.makedirs(eagle_prj_dir, exist_ok=True)
# This is an instruction to create the directory defined above

eagle_prj_dir
# This prints out the directory path name

The API Site for the 2017 Historical Fire Perimeters from the National Interagency Fire Center (NIFC) can be found at:

https://data-nifc.opendata.arcgis.com/datasets/historic-perimeters-2017/api

In [ ]:
# Access the data from NIFC website
eagle_gdf=gpd.read_file(
    "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis"
    "/rest/services/Historic_Geomac_Perimeters_2017"
    "/FeatureServer/0/query?where=incidentname%20%3D%20"
    "'EAGLE%20CREEK'%20AND%20latest%20%3D%20'Y'&outFields="
    "*&outSR=4326&f=json"
)

eagle_gdf

In [ ]:
# Location of Cascade Locks, OR 
# Citation: https://latitude.to/articles-by-country/us/united-states
# /44009/cascade-locks-oregon
casloc_lat = 45.6698
casloc_lon = -121.8906

# Create folium map of the Eagle Creek Fire boundary, Cascade Locks, OR
m = folium.Map(
    location = (casloc_lat, casloc_lon),        # Location to center display
    zoom_start = 10,                            # Set initial zoom level
    tiles = "Stamen Terrain",                   # Select basemap to use
    scrollWheelZoom=False                       # Turn off zoom while scrolling
)

# Put a marker at the city location
folium.Marker([casloc_lat, casloc_lon], popup="Cascade Locks, OR"
).add_to(m)

# Convert the GeoDataFrame to GeoJSON format
eagle_geojson = eagle_gdf.to_json()

# Creat a style for the GeoJSON output
eagle_json_style = {
        "fillColor":"red",
        "color":"black",
        "weight": 2,
        "dashArray": "5, 5",
}

# Add the GeoJSON data to the map as a GeoJSON layer
folium.GeoJson(
    eagle_geojson,
    style_function=lambda feature: eagle_json_style
).add_to(m)

# Display the map
m

Location map showing the total burn area of the 2017 Eagle Creek Fire in the Columbia River Gorge near Cascade Locks, Oregon

### **Assessing Vegetative Health using AQUA-MODIS-NDVI**

### Data Description:

Data for this analysis come from NASA's Application for Extracting and Exploring Analysis Ready Samples [AppEEARS](https://appeears.earthdatacloud.nasa.gov/) API portal. The selected layers are 250m 16-day NDVI values [(MYD13Q1.061)](https://lpdaac.usgs.gov/products/myd13q1v061/) taken for a three month interval covering peak summer vegetation (June 1 - August 31) for 2015-2020.

The data are a product of the AQUA MODIS satellite and sensor and contain normalized difference vegetation indices (NDVI). NDVI indices are useful for assessing "greenness" or vegetation health in a landscape. These quantitative indices are derived from the measure of light reflection and absorbance across multiple spectral bands which vary with vegetative health.

In [ ]:
# Download the NDVI data from NASA Appeears API
# Using data from Aqua MODIS https://appeears.earthdatacloud.nasa.gov/products
ndvi_downloader = etapp.AppeearsDownloader(
    download_key='modis-ndvi',          # Dataset name
    ea_dir=eagle_prj_dir,               # Name of project directory
    product='MYD13Q1.061',              # Product name from Aqua MODIS
    layer='_250m_16_days_NDVI',         # High resolution layer of interest
    start_date='06-01',                 # Recurring date - peak green start
    end_date='08-31',                   # Recurring date - peak green end
    recurring=True,                     # Indicates a recurring date used
    year_range=[2015,2020],             # Range of years to be extracted [list]
    polygon=eagle_gdf                   # Bounadry of the CAMP fire
)

# Download files if the download directory does not exist
if not os.path.exists(ndvi_downloader.data_dir):
    ndvi_downloader.download_files()

ndvi_downloader

In [ ]:
# Load all downloaded data files
ndvi_path_list = glob(os.path.join(
    ndvi_downloader.data_dir, '*', '*NDVI*.tif'))

# In the active directory, go in one folder level '*' then
# sort through the files, searching for files wiht NDVI

ndvi_path_list

In [ ]:
# Define variables
day_of_yr_start = -19           # start of the date in the name string
day_of_yr_end = -12             # End of the date in the name string
scale_factor = 10000            # NDVI between 1 and -1

# Import data from all NDVI files
ndvi_da_list = []
for ndvi_path in ndvi_path_list:
    # Get date from file name
    day_of_yr = ndvi_path[day_of_yr_start:day_of_yr_end] # Filename indices
    date = pd.to_datetime(day_of_yr, format='%Y%j')      # Date formatting

    # Open dataset - opens tiff file as a rioxarray (gridded data with coords)
    # Import no data as NAN. Squeeze data to remove extra dimension
    da = rxr.open_rasterio(ndvi_path, masked=True).squeeze()

    # Prepare to concatenate: Add date dimension and clean up metadata
    da = da.assign_coords({'date': date})               # Add date as coord
    da = da.expand_dims({'date': 1})                    # Add date as dimension
    da.name = 'NDVI'

    # Divide by scale factor
    da = da / scale_factor

    # Add the DataArray to the end of the accumulator list
    ndvi_da_list.append(da)

ndvi_da_list

In [ ]:
# Stack arrays by date into a time series
ndvi_ds = xr.combine_by_coords(ndvi_da_list, coords=['date'])
ndvi_ds

### Map showing change in NDVI values 2018-2016

In [ ]:
# Plot mean NDVI for 2018 spatially (rioxarray method)
ndvi_2018 = (ndvi_ds
 .sel(date='2018')                      # Specify  period of time to subset
 .mean('date')                          # Calculate mean NDVI
)

# Plot mean NDVI for 2016 spatially (rioarray method)
ndvi_2016 = (ndvi_ds
 .sel(date='2016')                      # Specify  period of time to subset
 .mean('date')                          # Calculate mean NDVI
)

# Calculate NDVI Difference
ndvi_diff = (ndvi_2018-ndvi_2016)

# Plot Differenced NDVI Data - use divergent colors - with boudary overlay
(ndvi_diff.hvplot(
    x='x', y='y',
    xlabel='Longitude (decimal degrees)', 
    ylabel='Latitude (decimal degrees)',
    title='Difference in NDVI between summer 2018-2016',
    cmap='PiYG',                        # Add _r to colormap name to reverse
    geo=True
)
*
eagle_gdf.hvplot(geo=True, fill_color='none')
)

In [ ]:
# Get envelop (boundary) of fire boundary polygon
out_gdf = gpd.GeoDataFrame(geometry=eagle_gdf.envelope)

# Get difference between fire boundary and envelop (geodataframe)
out_gdf.overlay(eagle_gdf, how="difference")

In [ ]:
# Clip data boundaries both inside and outside the camp fire area
ndvi_fire_ds = ndvi_ds.rio.clip(eagle_gdf.geometry)
ndvi_out_ds = ndvi_ds.rio.clip(out_gdf.geometry)

# To check that clipping is working, use an individual date
# ndvi_ds.rio.clip(camp_gdf.geometry).NDVI.sel(date='2019-05').plot()

# Can also check coordinate systems for both datasets
# ndvi_ds.rio.crs, camp_gdf.crs

In [ ]:
# Create dataframe with just mean NDVI values per year
ndvi_fire_df = (ndvi_fire_ds.groupby(ndvi_fire_ds.date.dt.year)
 .mean(...).NDVI.to_dataframe()[['NDVI']])

ndvi_out_df = (ndvi_out_ds.groupby(ndvi_out_ds.date.dt.year)
 .mean(...).NDVI.to_dataframe()[['NDVI']])

ndvi_fire_df, ndvi_out_df

# Get year for all layers in dataset and group by year to get individual years
# There are multiple variables in the dataset. When using mean, it needs to 
# know which variables to collapse. In this case, both x and y.
# mean('x', 'y') is simplifed to '...'. Take mean of NDVI and made dataframe
# Use [[]]to get rid of extra columns

In [ ]:
# Plot the difference between the area impacted and not impacted by fire
(ndvi_fire_df - ndvi_out_df).hvplot(
    xlabel='Year', ylabel='dNDVI',
    title="Difference in NVDI values within and around the Camp fire boundary"
)

# Edit this text

### Vegetation appears nearly recoverd two years after the Camp fire destroyed most of the area around Paradise, CA. 

 During the fire event, NDVI dropped by nearly 0.08 within the fire boundary area compared the region not impacted by the fire. NDVI values pre- and post-fire show very similar levels; however, these values do not indicate whether the recovery in vegetation is related to growth of scrub or forest. Most likely, this recovery represents "first growth" which usually includes flowers, bushes, and other small plants near the forest floor. Several more years will be required for the forest cover to recover.